## kubesession ##

`kubesession` is a very simple layer on top of the [requests](https://docs.python-requests.org/en/master/) library that makes it easy to call the [Kubernetes](https://k8s.io) API. 

Both requests *and* kubernetes have very well defined APIs, so any abstraction that papers over them will always be leaky without much benefit. There is benefit to automating some boilerplate code that is required to use the k8s API - mostly around authentication & parsing `.kube/config` files. 

This library provides such conveniences. In particular, it allows you to:

1. Use `kube://` URLs, which will automatically be translated to the appropriate server URL based on `.kube/config`
2. Deal with the case of connecting to IPs directly, which have a SSL certificate with a SAN for 'kubernetes'. This is common when dealing with minikube and other setups. TODO: Evaluate the security of this *properly*!
3. Deals with authentication properly, reading appropriate entries from `.kube/config`. Supports client certificate, token & basic auth.
4. Deals with default namespace set in `.kube/config` properly. When calling out to API methods, the string `{namespace}` will always be replaced with the actual default namespace.


TODO: Write example of calling out to k8s with raw requests, then with kubesession.


In [11]:
import requests
from requests.adapters import HTTPAdapter

class _KubernetesAdapter(HTTPAdapter):
    """
    A HTTPS Adapter for Python Requests that makes working with kubernetes API easier.

    It does the following special things:
        
        - Replace 'kube://' with the server URL of the kubernetes master.
          This allows your code to call URLs like 'kube:///api/v1' rather
          than interpolate the URL of the master in all the places.
        - Replace the literal string '{namespace}' with the namespace parameter
          passed in. Again, to avoid having to interpolate the default everywhere.
        - For SSL checks, assert the hostname against what is passed in the 
          Host: header, rather than whatever we are connecting to. This is only
          done if the Host: header is set explicitly. This should be used
          very carefully, since this could possibly be used as a security exploit.
          
    Do not use this directly, always call get_kubernetes_session instead.
    
    Adapted from the wonderful requests_toolbelt, specifically from
    https://github.com/sigmavirus24/requests-toolbelt/blob/master/requests_toolbelt/adapters/host_header_ssl.py
    """
    def __init__(self, server, namespace, **kwargs):
        super().__init__(**kwargs)
        self.server = server
        self.namespace = namespace

    def send(self, request, **kwargs):
        request.url = request.url.replace('kube://', self.server).format(
            namespace=self.namespace
        )
        host_header = None
        # HTTP headers are case-insensitive (RFC 7230)
        for header in request.headers:
            if header.lower() == "host":
                host_header = request.headers[header]
                break

        connection_pool_kwargs = self.poolmanager.connection_pool_kw

        if host_header:
            connection_pool_kwargs["assert_hostname"] = host_header
        elif "assert_hostname" in connection_pool_kwargs:
            # an assert_hostname from a previous request may have been left
            connection_pool_kwargs.pop("assert_hostname", None)

        return super().send(request, **kwargs)


In [12]:
from urllib.parse import urlparse
import ipaddress

def get_kube_session(config, current_context=None, current_namespace='default'):
    if current_context is None:
        current_context = config['current-context']
    
    context = [c for c in config['contexts'] if c['name'] == current_context][0]['context']
    cluster = [c for c in config['clusters'] if c['name'] == context['cluster']][0]['cluster']
    user = [u for u in config['users'] if u['name'] == context['user']][0]['user']

    
    s = requests.Session()
    s.mount('kube://', _KubernetesAdapter(cluster['server'], context.get('namespace', current_namespace)))
    
    # If we are using client certificates for authentication, set 'em!
    if 'client-certificate' in user:
        s.cert = (user['client-certificate'], user['client-key'])
        
    # TODO: Account for basic auth and token auth
    if cluster['server'].startswith('https://') and not cluster.get('insecure-skip-tls-verify', False):
        # If we are using https *and* connecting to an IP, attempt to
        # validate for the name 'kubernetes' rather than just the IP.
        parts = urlparse(cluster['server'])
        try:
            ipaddress.ip_address(parts.hostname)
            headers = {'Host': 'kubernetes'}
        except:
            # Not an IP address
            headers = {}
            raise
        s.headers = headers
        
        if 'certificate-authority' in cluster:
            s.verify = cluster['certificate-authority']
    return s

In [13]:
import yaml

with open('/home/yuvipanda/.kube/config') as f:
    s = get_kube_session(yaml.safe_load(f))
    print(s.get('kube:///api/v1/pods').json())

{'items': [{'status': {'hostIP': '10.0.2.15', 'phase': 'Running', 'conditions': [{'type': 'Initialized', 'lastProbeTime': None, 'lastTransitionTime': '2016-08-25T01:47:04Z', 'status': 'True'}, {'type': 'Ready', 'lastProbeTime': None, 'lastTransitionTime': '2016-08-25T01:48:45Z', 'status': 'True'}, {'type': 'PodScheduled', 'lastProbeTime': None, 'lastTransitionTime': '2016-08-25T01:47:04Z', 'status': 'True'}], 'podIP': '10.0.2.15', 'startTime': '2016-08-25T01:47:04Z', 'containerStatuses': [{'restartCount': 0, 'containerID': 'docker://2bddc88bb741d05119fe887208a23605a3ba6c556a7ed2129ea899f44463e04b', 'name': 'kube-addon-manager', 'image': 'gcr.io/google-containers/kube-addon-manager-amd64:v2', 'state': {'running': {'startedAt': '2016-08-25T01:48:44Z'}}, 'lastState': {}, 'imageID': 'docker://sha256:a876fb07f9c2bdf292436f8b4da8a975b7a9a2d2b39e00ddd9197b64d5ae34d0', 'ready': True}]}, 'metadata': {'uid': 'd304e62f-6a65-11e6-a25c-5256aad3edfb', 'name': 'kube-addon-manager-minikubevm', 'resour